<a href="https://colab.research.google.com/github/andreaderuvo/rc_car_tracking/blob/main/C8_test_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#settings
PROJECT = 'rc_car_tracking'
MODEL = 'yolo'

COLAB_BASE_PROJECT = f'/content/{PROJECT}'
DRIVE_MOUNT_FOLDER = 'drive'
DRIVE_BASE_PROJECT = f'{DRIVE_MOUNT_FOLDER}/MyDrive/rc_car_tracking'
DRIVE_VIDEOS_FOLDER = f'{DRIVE_BASE_PROJECT}/videos'
DRIVE_YOLO_TRAINED_WEIGHTS_FOLDER = f'{DRIVE_BASE_PROJECT}/{MODEL}_trained_weights'
DRIVE_YOLO_DATASET = f'{DRIVE_BASE_PROJECT}/{MODEL}_dataset'
DRIVE_YOLO_TEST = f'{DRIVE_BASE_PROJECT}/{MODEL}_test'

In [ ]:
#google drive
from google.colab import drive
drive.mount(DRIVE_MOUNT_FOLDER, force_remount=True)
!ln -s $DRIVE_BASE_PROJECT $COLAB_BASE_PROJECT

In [ ]:
#download and compile darknet enabling GPU mode
!rm -rf darknet
!git clone 'https://github.com/AlexeyAB/darknet.git'
%cd darknet
!rm Makefile
!wget https://raw.githubusercontent.com/andreaderuvo/rc_car_tracking/main/darknet/Makefile
!make
!chmod +x darknet
%cd ..

In [ ]:
!wget https://raw.githubusercontent.com/andreaderuvo/rc_car_tracking/main/darknet/cfg/yolov3_test.cfg

In [ ]:
!mkdir /content/predictions
%cd darknet

In [ ]:
#test on single image
!./darknet detector test /content/rc_car_tracking/yolo_dataset/labelled_data.data /content/yolov3_test.cfg /content/rc_car_tracking/yolo_trained_weights/yolov3_final.weights /content/rc_car_tracking/yolo_dataset/2_17_1651069556484.jpg -dont_show && mv predictions.jpg /content/predictions/output.jpg && rm predictions.jpg

In [ ]:
#test all images
!rm -rf /content/predictions && mkdir /content/predictions && cat /content/rc_car_tracking/yolo_dataset/test.txt | while read line; do ./darknet detector test /content/rc_car_tracking/yolo_dataset/labelled_data.data /content/yolov3_test.cfg /content/rc_car_tracking/yolo_trained_weights/yolov3_final.weights $line -dont_show && mv predictions.jpg /content/predictions/$(basename $line); done

In [ ]:
import os
import shutil
os.makedirs('rc_car_tracking/yolo_test', exist_ok=True)

for file in os.listdir('/content/predictions'):
  shutil.copy(f'/content/predictions/{file}', '/content/rc_car_tracking/yolo_test')